In [1]:
import time
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler 
from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.metrics import get_scorer_names

from sklearn.svm import LinearSVR, SVR

# Import helperfunctions
from ML_functions import fun_load_settings, fun_load_data, fun_preprocessing, fun_fit_tuning
from ML_functions import fun_convert_time
from ML_functions import fun_scaled_neg_MAPE, fun_tuning_results, fun_scores

# Set the default optimization problem for the case of manual executing the script (choose either "TSP" or "CVRP")
default_optimization_problem = "TSP"

# Call the function to define optimization_problem based on how the notebook is executed
# If the notebook is run by the script "main.ipynb", load optimization_problem from "settings.json". Otherwise use the default optimization problem from above
optimization_problem = fun_load_settings(default_optimization_problem)

# Load data and start the time count for the script within the function fun_load_data
data, start_script = fun_load_data(optimization_problem)

# Do the train test split during the preprocessing
X_train, X_test, y_train, y_test, train_data = fun_preprocessing(data, train_size=0.8)

The notebook was executed by another notebook. :)
Optimization problem: 'TSP'


# **Linear Support Vector Machine and SVM with Kernels - Kernel Machines**
### **Support Vector Machine (SVM)**
- the regularization parameter C controls trade-off between maximizing the margin and minimizing the classification error (how important it is to satisfy the constraint)

In [2]:
if (optimization_problem == "TSP"):

    # Define the model pipeline
    pipe = Pipeline(steps=[("scaler", None), 
                           ("SVM", LinearSVR(max_iter=1000, random_state=42))])

    param_grid = {"scaler": [StandardScaler(), MinMaxScaler()],
                  "SVM__C": [0.1, 1, 10, 50], 
                  "SVM__epsilon": [0.01, 0.1, 1]}

    grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, 
                               cv=GroupKFold(n_splits=3).split(X_train, y_train, groups=X_train.index.get_level_values(level="Instance ID")), 
                               scoring=fun_scaled_neg_MAPE, refit=False, verbose=True, n_jobs=-1)
    tuning_details = fun_fit_tuning(grid_search, X_train, y_train, file_name=f"{optimization_problem}_SVM")

    # Estimate model performance with cross-validation on the train set (scoring: MAPE and RMSE)
    model_results_dict = fun_scores(grid_search, X_train, y_train)
    model_results_dict.update(tuning_details)

    # Create a dictionary to store the results
    results_dict = {"Linear SVM": model_results_dict}

    # View grid search CV scores of all parameter combinations
    results_df = fun_tuning_results(grid_search, param_grid)

else: print("This cell is only executed for the TSP!")

Fitting 3 folds for each of 24 candidates, totalling 72 fits


{'Search type': 'GridSearchCV',
 'Parameter combinations': 24,
 'Total tuning time': '1m, 39s',
 'Total tuning fit time': '5m, 47s',
 'Total tuning prediction time': '2s'}

CV MAPE (scaled) train data: 19.18 %


**Best model / parameter combination:**

{'SVM__C': 0.1, 'SVM__epsilon': 0.01, 'scaler': MinMaxScaler()}

**Cross validation scores of different parameter combinations:**

,scaler,C,epsilon,mean_test_score,converted_mean_fit_time
0,MinMaxScaler(),0.1,0.01,-0.191797,0s
1,MinMaxScaler(),0.1,0.10,-0.191906,0s
2,MinMaxScaler(),1.0,0.01,-0.192572,1s
3,MinMaxScaler(),1.0,0.10,-0.192738,1s
4,MinMaxScaler(),50.0,0.01,-0.192874,20s
5,MinMaxScaler(),0.1,1.00,-0.192919,0s
6,MinMaxScaler(),1.0,1.00,-0.193086,1s
7,StandardScaler(),50.0,1.00,-0.193230,33s
8,StandardScaler(),0.1,0.10,-0.193320,1s
9,StandardScaler(),0.1,0.01,-0.193438,1s


### **Gaussian Kernel Machine**
- limit the number of iterations with "max_iter" to avoid excessive computation times (model may not reach convergence)
- increase "chace_size" in MB to store more data of the kernel matrix in memory
- gamma "scale" uses 1 / (n_features * X.var())
- gamma "auto" uses 1 / n_features

In [3]:
# Create pipline to scale each fold first during CV
pipe = Pipeline(steps=[("scaler", None), 
                       ("SVM", SVR(kernel="rbf", cache_size=3000, max_iter=10000))])

param_grid = {"scaler": [StandardScaler(), MinMaxScaler()], 
              "SVM__C": [10, 100, 200], 
              "SVM__gamma": ["scale", "auto"], # 0.001, 0.01, 0.1 did not perform
              "SVM__epsilon": [0.01, 0.1, 1]} # gamma_scale = 1 / (X_train.shape[1] * np.mean(np.var(X_train, axis=0)))

grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, 
                           cv=GroupKFold(n_splits=3).split(X_train, y_train, groups=X_train.index.get_level_values(level="Instance ID")), 
                           scoring=fun_scaled_neg_MAPE, refit=False, verbose=True, n_jobs=-1)
tuning_details = fun_fit_tuning(grid_search, X_train, y_train, file_name=f"{optimization_problem}_KM")

# Estimate model performance with cross validation on the train set (scoring: MAPE and RMSE)
model_results_dict = fun_scores(grid_search, X_train, y_train)
model_results_dict.update(tuning_details)

# Save results to dictionary (CVRP: create the dictionary first)
try: results_dict["Kernel Machine"] = model_results_dict
except: results_dict = model_results_dict

# View grid search CV scores of all parameter combinations
results_df = fun_tuning_results(grid_search, param_grid)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


{'Search type': 'GridSearchCV',
 'Parameter combinations': 36,
 'Total tuning time': '35m, 37s',
 'Total tuning fit time': '1h, 42m',
 'Total tuning prediction time': '36m, 22s'}

CV MAPE (scaled) train data: 5.58 %


**Best model / parameter combination:**

{'SVM__C': 100,
 'SVM__epsilon': 1,
 'SVM__gamma': 'scale',
 'scaler': MinMaxScaler()}

**Cross validation scores of different parameter combinations:**

,scaler,C,gamma,epsilon,mean_test_score,converted_mean_fit_time
0,MinMaxScaler(),100,scale,1.00,-0.055756,"2m, 54s"
1,MinMaxScaler(),10,scale,1.00,-0.057434,"2m, 54s"
2,MinMaxScaler(),10,scale,0.10,-0.058098,"3m, 4s"
3,MinMaxScaler(),200,scale,1.00,-0.058446,"3m, 2s"
4,MinMaxScaler(),10,scale,0.01,-0.060025,"3m, 11s"
5,MinMaxScaler(),200,scale,0.10,-0.062164,"2m, 57s"
6,StandardScaler(),100,scale,0.01,-0.063220,"2m, 33s"
7,StandardScaler(),100,auto,0.01,-0.063220,"2m, 40s"
8,MinMaxScaler(),200,scale,0.01,-0.063810,"2m, 55s"
9,MinMaxScaler(),100,scale,0.10,-0.064566,"2m, 40s"


# **Compare Results**

In [4]:
if (optimization_problem == "TSP"):
    display(pd.DataFrame(results_dict).sort_values(by="MAPE", axis=1))
    print("Total script computation time:", fun_convert_time(start=start_script, end=time.time()))

,Kernel Machine,Linear SVM
MAPE,5.58,19.18
RMSE,None,None
CV computation time,None,None
Search type,GridSearchCV,GridSearchCV
Parameter combinations,36,24
Total tuning time,"35m, 37s","1m, 39s"
Total tuning fit time,"1h, 42m","5m, 47s"
Total tuning prediction time,"36m, 22s",2s


Total script computation time: 37m, 56s
